In [43]:
import torch

if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    # The correct way to get memory in PyTorch
    total_mem = torch.cuda.get_device_properties(0).total_memory
    
    print(f"✅ GPU Found: {device_name}")
    print(f"Memory: {total_mem / 1024**3:.2f} GB")
else:
    print("❌ GPU NOT found. Check your drivers or Secure Boot.")

✅ GPU Found: NVIDIA GeForce RTX 2050
Memory: 3.68 GB


In [44]:
# # Vocabulary Analysis (Optional)
# # Count token frequencies in the English subset
# token_counts = Counter()

# print("Analyzing vocabulary usage...")
# # Sample a subset if dataset is huge, but 300k is manageable or take first 10k
# for i, example in enumerate(dataset['train']):
#     if i > 10000: break # Analyze first 10k samples for speed
#     ids = tokenizer(example['source_text'], truncation=False, add_special_tokens=False)['input_ids']
#     token_counts.update(ids)

# print(f"Total unique tokens used (in sample): {len(token_counts)}")
# print(f"Vocab size of tokenizer: {tokenizer.vocab_size}")

# # Top 20 tokens
# top_20 = token_counts.most_common(20)
# print("Top 20 tokens:")
# for tid, count in top_20:
#     print(f"{tokenizer.decode([tid])}: {count}")

# # Tokens not used
# unused_count = tokenizer.vocab_size - len(token_counts)
# print(f"Unused tokens (approx): {unused_count} ({unused_count/tokenizer.vocab_size:.2%})")


# PII Masking Model Pipeline (Browser Optimized)

This notebook implements a pipeline to create a small, efficient PII masking model using the Lottery Ticket Hypothesis (LTH) and Quantization.

**Key Steps:**
1. **Load Data**: `ai4privacy/pii-masking-300k` (Filter for **English** only).
2. **Preprocessing**: Robust tokenization using character offsets to handle dataset quirks.
3. **Save Initial Weights**: Critical for LTH "rewinding".
4. **Fine-tune** -> **Prune** -> **Reset** -> **Retrain** -> **Quantize**.


In [45]:
# !pip install transformers datasets torch optimum onnxruntime onnx scikit-learn numpy pandas accelerate evaluate seqeval

In [46]:
import os
import json
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from torch.nn.utils import prune
from collections import Counter

# Configuration
MODEL_CHECKPOINT = "distilbert-base-uncased"
DATASET_NAME = "ai4privacy/pii-masking-300k"
OUTPUT_DIR = "./pii_model_output"
INITIAL_WEIGHTS_PATH = os.path.join(OUTPUT_DIR, "initial_weights.pt")

os.makedirs(OUTPUT_DIR, exist_ok=True)

## 1. Data Loading & Splits
We load the dataset and filter for **English** (`language == 'English'`).
Since the dataset only provides `train` and `validation` splits, we split `validation` into disjoint `validation` and `test` sets (50/50).

In [47]:
# Load Dataset
# Note: 'trust_remote_code=True' needed for some HF datasets, beneficial to keep.
dataset = load_dataset(DATASET_NAME, trust_remote_code=True)

# Filter for English. Verified key is 'language' == 'English'.
dataset = dataset.filter(lambda x: x["language"] == "English")

# Check splits. If 'test' exists, use it. Else split validation.
if "test" not in dataset:
    print("Creating Test split from Validation...")
    val_test_split = dataset["validation"].train_test_split(test_size=0.5, seed=42)
    dataset["validation"] = val_test_split["train"]
    dataset["test"] = val_test_split["test"]

print(dataset)


`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'ai4privacy/pii-masking-300k' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Creating Test split from Validation...
DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 29908
    })
    validation: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 3973
    })
    test: Dataset({
        features: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_rows: 3973
    })
})


## 2. Parsing & Label Extraction
The dataset stores complex fields (`privacy_mask`) as JSON strings (in some versions) or list objects. We ensure they are parsed and extract unique labels.

In [48]:
def parse_dataset_row(example):
    if isinstance(example['privacy_mask'], str):
        example['privacy_mask'] = json.loads(example['privacy_mask'])
    return example

# We map once. This is cached by HF datasets so subsequent calls are fast.
dataset = dataset.map(parse_dataset_row, num_proc=4)

print("Extracting unique labels from Train split...")
unique_labels = set()
# Use a generator for memory efficiency
for privacy_mask in dataset['train']['privacy_mask']:
    for entity in privacy_mask:
        unique_labels.add(entity['label'])

label_list = ["O"]
for label in sorted(list(unique_labels)):
    label_list.append(f"B-{label}")
    label_list.append(f"I-{label}")

label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}
print(f"Unique labels: {len(label_list)}")


Extracting unique labels from Train split...
Unique labels: 57


## 3. Robust Tokenization & Alignment
We use `return_offsets_mapping=True` to map directly from character spans in `privacy_mask` to tokenizer tokens.

In [49]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["source_text"],
        truncation=True,
        max_length=512,
        stride=64,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding=False # Dynamic padding in DataCollator
    )

    labels = []
    sample_mapping = tokenized_inputs.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_inputs.pop("offset_mapping")

    for i, offsets in enumerate(offset_mapping):
        sample_idx = sample_mapping[i]
        mask_list = examples["privacy_mask"][sample_idx]
        
        # Initialize with O
        chunk_labels = [label2id["O"]] * len(tokenized_inputs["input_ids"][i])
        
        for entity in mask_list:
            label_type = entity["label"]
            start_char = entity["start"]
            end_char = entity["end"]
            
            b_id = label2id.get(f"B-{label_type}")
            i_id = label2id.get(f"I-{label_type}")
            if b_id is None: continue
            
            # Find all overlapping tokens for this entity in this chunk
            # Overlap condition: not (token_end <= entity_start or token_start >= entity_end)
            overlapping_indices = []
            for idx, (t_start, t_end) in enumerate(offsets):
                if t_start == 0 and t_end == 0: continue
                if not (t_end <= start_char or t_start >= end_char):
                    overlapping_indices.append(idx)
            
            # Assign Labels
            for k, idx in enumerate(overlapping_indices):
                current_label = chunk_labels[idx]
                # Don't overwrite existing B/I labels unless they are O or -100
                if current_label != label2id["O"] and current_label != -100:
                     continue
                
                # Logic: The FIRST overlapping token gets B, others I.
                # UNLESS the entity actually started before this token (Sliding window split).
                if k == 0:
                    t_start = offsets[idx][0]
                    # If entity start is >= token start, it means this token contains the start -> B
                    if start_char >= t_start:
                        chunk_labels[idx] = b_id
                    else:
                        # Entity started before this token -> I
                        chunk_labels[idx] = i_id
                else:
                    chunk_labels[idx] = i_id
        
        # Mask special tokens
        for idx, (t_start, t_end) in enumerate(offsets):
            if t_start == 0 and t_end == 0:
                chunk_labels[idx] = -100
        
        labels.append(chunk_labels)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(
    tokenize_and_align_labels, 
    batched=True, 
    remove_columns=dataset["train"].column_names,
    num_proc=4
)


## 4. Save Initial Weights
Save the untrained (or pre-trained base) weights to support the Lottery Ticket Hypothesis rewinding step later.

In [50]:
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_CHECKPOINT, 
    # Pass the new label map
    num_labels=len(label_list), 
    id2label=id2label, 
    label2id=label2id
)

torch.save(model.state_dict(), INITIAL_WEIGHTS_PATH)
print(f"Initial weights saved to {INITIAL_WEIGHTS_PATH} with {len(label_list)} labels and sliding window configuration.")


Loading weights: 100%|██████████| 100/100 [00:00<00:00, 266.58it/s, Materializing param=distilbert.transformer.layer.5.sa_layer_norm.weight]   
DistilBertForTokenClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.bias    | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
classifier.bias         | MISSING    | 
classifier.weight       | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Initial weights saved to ./pii_model_output/initial_weights.pt with 57 labels and sliding window configuration.


In [51]:
# Evaluation Metrics
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [52]:
def evaluate_accuracy(model, dataset, batch_size=16):
    print("Evaluating model accuracy...")
    
    # Use a separate Trainer for evaluation to leverage the DataCollator and efficient batching
    eval_trainer = Trainer(
        model=model,
        args=TrainingArguments(output_dir="/tmp/eval", per_device_eval_batch_size=batch_size, report_to="none"),
        data_collator=DataCollatorForTokenClassification(tokenizer),
        eval_dataset=dataset,
        compute_metrics=compute_metrics
    )
    
    metrics = eval_trainer.evaluate()
    print("Evaluation Results:", metrics)
    return metrics


In [53]:
# Run Baseline Evaluation
# print("Running baseline evaluation on Test set...")
# baseline_metrics = evaluate_accuracy(model, tokenized_datasets["test"])
# print(f"Baseline F1: {baseline_metrics['eval_f1']:.4f}")


In [54]:
# 6. Class Imbalance Handling
from torch.nn import CrossEntropyLoss

def compute_class_weights(dataset, label2id):
    print("Computing class weights...")
    label_counts = torch.zeros(len(label2id))
    
    # Iterate over train set to count labels
    # Note: This might be slow if we iterate python-side.
    # A faster way is roughly estimating or using the already known statistics if available.
    # But let's do a quick pass or use a subset if needed. Here we do full pass.
    for i, example in enumerate(dataset):
        labels = example['labels']
        for label in labels:
            if label != -100:
                label_counts[label] += 1
    
    print(f"Label counts: {label_counts}")
    
    # Inverse frequency with smoothing
    weights = 1.0 / (label_counts + 100)
    weights = weights / weights.sum() * len(label2id) # Normalize
    
    # Clamp to avoid extreme weights (e.g. for O vs rare B-tags)
    weights = torch.clamp(weights, min=0.1, max=10.0)
    weights[label2id["O"]] = 1.0
    
    return weights

# Compute weights using the training set
class_weights = compute_class_weights(tokenized_datasets['train'], label2id)
print("Class weights calculated.", class_weights)


Computing class weights...
Label counts: tensor([2.5629e+06, 8.4220e+03, 3.4713e+04, 7.3680e+03, 4.1920e+03, 5.0000e+00,
        1.5000e+01, 7.6260e+03, 1.3317e+04, 5.8470e+03, 2.9440e+03, 6.7600e+03,
        3.0992e+04, 8.6380e+03, 6.5280e+04, 9.7310e+03, 8.4153e+04, 6.9700e+02,
        7.1920e+03, 7.8590e+03, 1.2035e+04, 2.0920e+03, 3.2460e+03, 9.5460e+03,
        5.1252e+04, 8.0970e+03, 1.3896e+05, 9.0170e+03, 1.7882e+04, 2.2550e+03,
        4.5170e+03, 7.2600e+02, 1.5370e+03, 5.5670e+03, 3.4585e+04, 8.9760e+03,
        3.9170e+04, 7.4300e+03, 2.0002e+04, 3.1090e+03, 5.4890e+03, 7.4680e+03,
        2.2150e+03, 9.5650e+03, 5.7162e+04, 7.5610e+03, 1.7520e+03, 7.3270e+03,
        1.7878e+04, 7.2990e+03, 5.4982e+04, 1.4625e+04, 3.1667e+04, 7.5240e+03,
        2.7760e+03, 1.1084e+04, 5.4005e+04])
Class weights calculated. tensor([ 1.0000,  0.2348,  0.1000,  0.2679,  0.4662, 10.0000, 10.0000,  0.2590,
         0.1491,  0.3364,  0.6573,  0.2917,  0.1000,  0.2290,  0.1000,  0.2035,
        

In [55]:
# 7. Custom Weighted Trainer
data_collator = DataCollatorForTokenClassification(tokenizer)
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        # Use class weights injected into the loss function
        loss_fct = CrossEntropyLoss(weight=class_weights.to(model.device), ignore_index=-100)
        loss = loss_fct(logits.view(-1, len(label2id)), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss


In [ ]:
# 8. Training Configuration & Execution
from transformers import EarlyStoppingCallback
import torch
import shutil

# Verify GPU
if torch.cuda.is_available():
    print(f"🚀 Training on GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️  WARNING: Training on CPU. This will be slow!")

args = TrainingArguments(
    OUTPUT_DIR,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    
    learning_rate=2e-5, # Lower LR for fine-tuning
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    num_train_epochs=4,
    
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2, # Effective batch size 32
    
    weight_decay=0.01,
    fp16=torch.cuda.is_available(), # Use mixed precision if GPU available
    dataloader_num_workers=4,
    logging_steps=100,
    report_to="none", # Change to wandb/tensorboard if needed
    seed=42
)

trainer = WeightedTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("Starting Training...")
try:
    trainer.train()
    print("✅ Training completed successfully.")
    trainer.save_model(os.path.join(OUTPUT_DIR, "final_model"))
except KeyboardInterrupt:
    print("\n🛑 Training interrupted by user. Saving checkpoint...")
    trainer.save_model(os.path.join(OUTPUT_DIR, "interrupted_checkpoint"))
    print("Checkpoint saved.")
except Exception as e:
    print(f"\n❌ info: Training failed with error: {e}")
    # Attempt to save despite error
    try:
        trainer.save_model(os.path.join(OUTPUT_DIR, "failed_checkpoint"))
        print("Crash checkpoint saved.")
    except:
        print("Could not save crash checkpoint.")
    raise e


warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


🚀 Training on GPU: NVIDIA GeForce RTX 2050
Starting Training...


## 6. Pruning & Quantization Logic
Placeholders for subsequent steps.

In [ ]:
# Pruning Placeholder
# ...

# Quantization Placeholder
# ...

In [ ]:
# # Verification Step
# print("Verifying data format...")
# if len(tokenized_datasets['train']) > 0:
#     sample = tokenized_datasets['train'][0]
#     print("Keys:", sample.keys())
#     print("Input IDs length:", len(sample['input_ids']))
#     print("Labels length:", len(sample['labels']))
    
#     # Check for -100
#     print("Labels sample (first 20):", sample['labels'][:20])
    
#     # Check for valid IDs
#     import numpy as np
#     labels = np.array(sample['labels'])
#     print("Unique labels in sample:", np.unique(labels))
    
#     # Check for any label ID that is not in label2id and not -100
#     valid_ids = set(label2id.values())
#     valid_ids.add(-100)
#     invalid = [l for l in labels if l not in valid_ids]
#     if invalid:
#         print(f"WARNING: Found invalid label IDs: {invalid}")
#     else:
#         print("All label IDs are valid.")
# else:
#     print("Training set is empty!")
